In [ ]:
# Reference tutorial: https://www.thepythoncode.com/article/text-classification-using-tensorflow-2-and-keras-in-python

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

from glob import glob
import random
import os

In [ ]:
dataFrame = pd.read_csv("gdrive/My Drive/2020_SDP/Project_MaPPIng_Phase II/Tech_Team/policy_dataset.csv", encoding='UTF-8')
dataFrame.head()

,_id,url,body,title_google,main_query,aux_query,validation_flag
0,5ec0e7c54e24cd5aeeb9ae69,https://www.daily-sun.com/post/474802/Corona-h...,The under-construction 660MW coal-fired power ...,title,"""""""""""","[""covid"",""Corona"",""Coronavirus"",""COVID-19""]",1
1,5ec0e7cc4e24cd5aeeb9ae6a,https://www.newagebd.net/article/102134/payra-...,The country’s largest Payra 1320 MW Thermal Po...,title,"""""""""""","[""covid"",""Corona"",""Coronavirus"",""COVID-19""]",1
2,5ec0e7cc4e24cd5aeeb9ae6b,https://www.airport-technology.com/news/covid-...,Visit our Covid-19 microsite for the latest co...,title,"""""""""""","[""covid"",""Corona"",""Coronavirus"",""COVID-19""]",1
3,5ec0e7cd4e24cd5aeeb9ae6c,https://global.chinadaily.com.cn/a/202003/26/W...,By David Ho in Hong Kong | China Daily Global ...,title,"""""""""""","[""covid"",""Corona"",""Coronavirus"",""COVID-19""]",1
4,5ec0e7ce4e24cd5aeeb9ae6d,http://www.fredericton.ca/en/news/city-hall/ci...,Fredericton City Council has decided to postpo...,title,"""""""""""","[""covid"",""Corona"",""Coronavirus"",""COVID-19""]",1


In [ ]:
train_set = pd.read_csv("gdrive/My Drive/SJ/data/train_set.tsv", sep="\t", encoding='UTF-8', names=['original_text', 'validation_flag', 'body'])
train_set.head()

,original_text,validation_flag,body
0,1,1,the countrys largest payra mw thermal power pl...
1,1,1,the countrys largest payra mw thermal power pl...
2,1,1,the countrys largest payra mw thermal power pl...
3,1,1,the countrys largest payra mw thermal power pl...
4,1,1,the countrys largest payra mw thermal power pl...


In [ ]:
val_set = pd.read_csv('gdrive/My Drive/SJ/data/val_set.tsv', sep='\t', encoding='UTF-8', names=['original_text', 'validation_flag', 'body'])
val_set.head()

,original_text,validation_flag,body
0,0,1,the under construction mw coal fired power pla...
1,0,1,the under construction coal fired power plant ...
2,0,1,the under construction mw coal fired power pla...
3,0,1,the under construction mw ember fired power em...
4,0,1,the under construction mw coal fired power pla...


In [ ]:
# If the data is already splited into train_set and val_set
def load_data(train_set, val_set, num_words, sequence_length, oov_token = None):
    
    # read articles of the trained set
    train_articles = []
    for text in train_set['body']:
      text = text.strip()
      train_articles.append(text)
    
    # read flags of the trained set
    train_flags = []
    for label in train_set['validation_flag']:
      train_flags.append(label)

    # read articles of the validation set
    val_articles = []
    for text in val_set['body']:
      text = text.strip()
      val_articles.append(text)
    
    #read articcles of the validation set
    val_flags = []
    for label in val_set['validation_flag']:
      val_flags.append(label)

    # tokenize the dataset corpus, delete uncommon words such as names, etc.
    tokenizer = Tokenizer(num_words = num_words, oov_token = oov_token)
    tokenizer.fit_on_texts(train_articles)
    X_train = tokenizer.texts_to_sequences(train_articles)
    X_train, y_train = np.array(X_train), np.array(train_flags)
    # pad sequences with 0's
    X_train = pad_sequences(X_train, maxlen=sequence_length)
    # convert labels to one-hot encoded
    y_train = to_categorical(y_train)

    # transform your validation data the same way you transform your training data
    # (since the data is already splitted into the train sets and validation sets)
    tokenizer.fit_on_texts(val_articles)
    X_val = tokenizer.texts_to_sequences(val_articles)
    X_val, y_val = np.array(X_val), np.array(val_flags)
    X_val = pad_sequences(X_val, maxlen=sequence_length)
    y_val = to_categorical(y_val)

    # # split data to training and testing sets
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 1)
    
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_val
    data["y_train"] = y_train
    data["y_test"] = y_val
    data["tokenizer"] = tokenizer
    data["int2label"] =  {0: "negative", 1: "positive"}
    data["label2int"] = {"negative": 0, "positive": 1}
    
    return data

In [ ]:
# If the data is not splitted
def load_data(dataFrame, num_words, test_size, sequence_length, oov_token = None):
    
    # read articles
    articles = []
    for text in dataFrame['body']:
      text = text.strip()
      articles.append(text)
    
    # read flags
    flags = []
    for label in dataFrame['validation_flag']:
      flags.append(label)

    # tokenize the dataset corpus, delete uncommon words such as names, etc.
    tokenizer = Tokenizer(num_words = num_words, oov_token = oov_token)
    tokenizer.fit_on_texts(articles)
    X = tokenizer.texts_to_sequences(articles)
    X, y = np.array(X), np.array(flags)

    # pad sequences with 0's
    X = pad_sequences(X, maxlen=sequence_length)

    # convert labels to one-hot encoded
    y  = to_categorical(y)

    # split data to training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 1)
    
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_test
    data["y_train"] = y_train
    data["y_test"] = y_test
    data["tokenizer"] = tokenizer
    data["int2label"] =  {0: "negative", 1: "positive"}
    data["label2int"] = {"negative": 0, "positive": 1}
    
    return data

In [ ]:
# get pre-trained GloVe word vectors, which are pre-trained vectors that map each word to a vector of a specific size.
# The size parameter is often called embedding size, although GloVe uses 50, 100, 200 or 300 embedding size vectors.

def get_embedding_vectors(word_index, embedding_size=200):
  embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
  with open(f"gdrive/My Drive/SJ/glove.6B/glove.6B.{embedding_size}d.txt", encoding="utf8") as f:
    for line in tqdm(f, "Reading GloVe"):
      values = line.split()
      # get the word as the first word in the line
      word = values[0]
      if word in word_index:
          idx = word_index[word]
          # get the vectors as the remaining values in the line
          embedding_matrix[idx] = np.array(values[1:], dtype="float32")
  return embedding_matrix

In [ ]:
def create_model(word_index, units=128, n_layers=1, cell=LSTM, bidirectional=False,
                 embedding_size=100, sequence_length=100, dropout=0.3, 
                 loss="categorical_crossentropy", optimizer="adam", 
                 output_length=2):
  # Constructs a RNN model given its parameters
  embedding_matrix = get_embedding_vectors(word_index, embedding_size)
  model = Sequential()
  
  # add the embedding layer
  model.add(Embedding(len(word_index) + 1, embedding_size, weights=[embedding_matrix],
                      trainable=False, input_length=sequence_length))
  for i in range(n_layers):
    if i == n_layers - 1:
        # last layer
        if bidirectional:
            model.add(Bidirectional(cell(units, return_sequences=False)))
        else:
            model.add(cell(units, return_sequences=False))
    else:
        # first layer or hidden layers
        if bidirectional:
            model.add(Bidirectional(cell(units, return_sequences=True)))
        else:
            model.add(cell(units, return_sequences=True))
    model.add(Dropout(dropout))
  model.add(Dense(output_length, activation="softmax"))
  # compile the model
  model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
  return model

In [ ]:
# max number of words in each sentence
SEQUENCE_LENGTH = 300

# N-Dimensional GloVe embedding vectors
EMBEDDING_SIZE = 300

# number of words to use, discarding the rest
N_WORDS = 10000

# out of vocabulary token
OOV_TOKEN = None

# 30% testing set, 70% training set
TEST_SIZE = 0.3

# number of CELL layers
N_LAYERS = 1

# the RNN cell to use, LSTM in this case
RNN_CELL = LSTM

# whether it's a bidirectional RNN
IS_BIDIRECTIONAL = True

# number of units (RNN_CELL ,nodes) in each layer
UNITS = 64

# dropout rate
DROPOUT = 0.3

### Training parameters
LOSS = "categorical_crossentropy"
OPTIMIZER = "adam"
BATCH_SIZE = 64
EPOCHS = 6

def get_model_name(dataset_name):
  # construct the unique model name
  model_name = f"{dataset_name}-{RNN_CELL.__name__}-seq-{SEQUENCE_LENGTH}-em-{EMBEDDING_SIZE}-w-{N_WORDS}-layers-{N_LAYERS}-units-{UNITS}-opt-{OPTIMIZER}-BS-{BATCH_SIZE}-d-{DROPOUT}"
  if IS_BIDIRECTIONAL:
    # add 'bid' str if bidirectional
    model_name = "bid-" + model_name
  if OOV_TOKEN:
    # add 'oov' str if OOV token is specified
    model_name += "-oov"
  return model_name

In [ ]:
# create these folders if they does not exist
main_dir = "gdrive/My Drive/SJ/"
if not os.path.isdir(main_dir + "results"):
  os.mkdir(main_dir + "results")
if not os.path.isdir(main_dir + "logs"):
  os.mkdir(main_dir + "logs")
if not os.path.isdir(main_dir + "data"):
  os.mkdir(main_dir + "data")

In [ ]:
# # load the data - when the data is splitted
# data = load_data(train_set, val_set, N_WORDS, SEQUENCE_LENGTH, oov_token=OOV_TOKEN)

# load the data - when the data is not splitted
data = load_data(dataFrame, N_WORDS, TEST_SIZE, SEQUENCE_LENGTH, oov_token=OOV_TOKEN)

# construct the model
model = create_model(data["tokenizer"].word_index, units=UNITS, n_layers=N_LAYERS, 
                    cell=RNN_CELL, bidirectional=IS_BIDIRECTIONAL, embedding_size=EMBEDDING_SIZE, 
                    sequence_length=SEQUENCE_LENGTH, dropout=DROPOUT, 
                    loss=LOSS, optimizer=OPTIMIZER, output_length=data["y_train"][0].shape[0])
model.summary()

model_name = get_model_name("policy_dataset")

# using tensorboard on 'logs' folder
tensorboard = TensorBoard(log_dir=os.path.join(main_dir + "logs", model_name))

# start training
history = model.fit(data["X_train"], data["y_train"],
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(data["X_test"], data["y_test"]),
                    callbacks=[tensorboard],
                    verbose=1)

# save the resulting model into 'results' folder
model.save(os.path.join(main_dir + "results", model_name) + ".h5")

Reading GloVe: 400000it [00:17, 23062.35it/s]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          5388600   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               186880    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 5,575,738
Trainable params: 187,138
Non-trainable params: 5,388,600
_________________________________________________________________
Epoch 1/6
3/3 [==============================] - 2s 556ms/step - loss: 0.7068 - accuracy: 0.4929 - val_loss: 0.6490 - val_accuracy: 0.6500
Epoch 2/6
3/3 [==============================] - 0s 50ms/step - loss: 0.5804 - accuracy: 0.7286 - v

In [ ]:
import pickle

# saving the tokenizer
with open('gdrive/My Drive/SJ/tocknizer.pickle', 'wb') as handle:
    pickle.dump(data['tokenizer'], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# load and evaluate a saved model
from numpy import loadtxt
import tensorflow as tf
 
# load model
model = tf.keras.models.load_model('gdrive/My Drive/SJ/results/bid-policy_dataset-LSTM-seq-300-em-300-w-10000-layers-1-units-64-opt-adam-BS-64-d-0.3.h5')

# max number of words in each sentence
SEQUENCE_LENGTH = 300

# N-Dimensional GloVe embedding vectors
EMBEDDING_SIZE = 200

# number of words to use, discarding the rest
N_WORDS = 10000

# out of vocabulary token
OOV_TOKEN = None

# 30% testing set, 70% training set
TEST_SIZE = 0.3

# load the data - when the data is not splitted
data = load_data(dataFrame, N_WORDS, TEST_SIZE, SEQUENCE_LENGTH, oov_token=OOV_TOKEN)

def get_predictions(text):
  sequence = data["tokenizer"].texts_to_sequences([text])
  # pad the sequences
  sequence = pad_sequences(sequence, maxlen=SEQUENCE_LENGTH)
  # get the prediction
  prediction = model.predict(sequence)[0]
  return prediction, data["int2label"][np.argmax(prediction)]

In [ ]:
def get_predictions(text):
  sequence = data["tokenizer"].texts_to_sequences([text])
  # pad the sequences
  sequence = pad_sequences(sequence, maxlen=SEQUENCE_LENGTH)
  # get the prediction
  prediction = model.predict(sequence)[0]
  return prediction, data["int2label"][np.argmax(prediction)]

In [ ]:
text = "Completion of the 150-km Tanakpur-Pithoragarh stretch of the chardham all-weather road project will be delayed by nearly six months due to the shortage of labourers amid the ongoing lockdown, an official said on Thursday. 'This particular stretch of the project was to be completed in June this year. However, it is not likely to be ready before November now as all migrant labourers working on the project have left since their exodus began with the announcement of the lockdown,' NHAI Executive Engineer L D Mathela who is in charge of the project said. The shortage of labourers and technical staff has affected cutting, hot mixing, crushing and quarrying work of the project, the National Highways Authority of India (NHAI) official said. Even if work on the project resumes after May 3 it is bound to be affected by the onset of monsoon in June-July, he further said. Frequent rain and landslides during monsoon often lead to suspension of work on the project slowing its progress, Mathela said. The chardham all-weather road is a pet project of Prime Minister Narendra Modi who keeps monitoring its progress personally."
output_vector, prediction = get_predictions(text)
print("Output vector:", output_vector)
print("Prediction:", prediction)

Output vector: [0.4810909  0.51890904]
Prediction: positive


In [ ]:
text = "Looks like Taylor Swift and Ariana Grande were right to caution their fans about the coronavirus pandemic: Everyone who is able to stay home amid the spread of the COVID-19 virus should do so. Not even your favorite celebrities are immune. Since Tom Hanks and Rita Wilson confirmed their diagnoses in Australia, the question became a matter of when other famous faces would come down with the virus next, not if. Even Heidi Klum, who says she could not get a hold of a test, revealed she’d been experiencing symptoms. Here’s every celebrity who’s said they tested positive for coronavirus:"
output_vector, prediction = get_predictions(text)
print("Output vector:", output_vector)
print("Prediction:", prediction)

Output vector: [0.9787909  0.02120909]
Prediction: negative
